In [ ]:
json_output_prompt = '''Return the response strictly in JSON.
Do not include explanations or markdown.

Schema:
{
  "intent": string,
  "confidence": number,
  "entities": string[]
  "output": string
}

If unsure, return null values.
'''

In [13]:
from pydantic import BaseModel
from typing import List, Optional

class ResponseModel(BaseModel):
    intent: Optional[str]
    confidence: Optional[float]
    entities: Optional[List[str]]
    output: str

In [4]:
from openai import OpenAI
from collections import defaultdict
import json
import requests

In [17]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
MODEL = "llama3.2"
OLLAMA_URL = "http://localhost:11434/api/generate"
input_prompts = [
    "what is the mail and female population of mumbai",
    "book a table for two at an italian restaurant tomorrow night",
    "tell me a joke about programmers",
    "what's the weather like in new york city today",
    "how do i make a chocolate cake from scratch",
    "which countries are in the european union",
    "how many contries are in asian continent",
    "how many galzaxies in the universe and which galaxies are closest to the milky way",
    "which better mumbai or pune or banglore for living",
    "who won the fifa world cup in 2018",
    "Rate Narendra modi and amit shah",
]
success = 0

for input_prompt in input_prompts:
    payload = {
            "model": MODEL,
            "prompt": json_output_prompt + input_prompt,
            "stream": False
        }

    response = requests.post(OLLAMA_URL, json=payload)
    response.raise_for_status()
    output = response.json()["response"]
    try:
        ResponseModel.model_validate_json(output)
        success += 1
    except Exception as e:
        print("exception occurred", e)
        pass
    
    print(f"{input_prompt} ---> {output}")

print(f"Success rate: {success}/{len(input_prompts)}")

exception occurred 1 validation error for ResponseModel
output
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
what is the mail and female population of mumbai ---> {"intent":"Unknown Intent","confidence":0,"entities":["Mumbai"],"output":null}
exception occurred 1 validation error for ResponseModel
output
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
book a table for two at an italian restaurant tomorrow night ---> {"intent":"book_table","confidence":null,"entities":["Italian","tomorrow","night"],"output":null}
tell me a joke about programmers ---> {"intent":"joke","confidence":0,"entities":[],"output":"Why do programmers prefer dark mode? Because light attracts bugs."}
what's the weather like in new york city today ---> {"intent":"weather_